## Image selecting from json file

In [ ]:
from pycocotools.coco import COCO
import requests

base = '/home/cvmlserver/Seohyeon/v-coco/'

coco = COCO('../data/annotations/raw/captions_val2014.json')

imgIds = coco.getImgIds()

images = coco.loadImgs(sorted(imgIds))
count = 0
past_count = 0
gender_word = ['person', 'man', 'woman', 'boy', 'girl']

for im in images[:]:
    annos = coco.loadAnns(coco.getAnnIds(imgIds=im['id']))
    download = False
    score = 0
    for anno in annos:
        caption = anno['caption'].lower().split(' ')
        if any(word in caption for word in gender_word):
            score += 1
        else:
            break

    if score == len(annos):
        download = True
    
    if download:
        count += 1
        img_data = requests.get(im['coco_url']).content
        with open(base+'/data/original/' + im['file_name'], 'wb') as handler:
            handler.write(img_data)

    if count % 100 == 0 and past_count != count:
        past_count = count
        print(count, 'images downloaded')
    if count == 1000:
        break

print(count, 'total images downloaded')


## Caption extracting and writing

In [5]:
from pycocotools.coco import COCO
import requests
import re
import os
import json

# change this path to your coco dataset path
coco = COCO('data/annotations/raw/captions_val2014.json')

imgIds = coco.getImgIds()

images = coco.loadImgs(sorted(imgIds))
count = 0

image_list = os.listdir('./data_new')

dict = {}

for name in image_list:
    name = name.split('.')[0]
    dict[name] = []
    for im in images:
        if name == im['file_name'].split('.')[0]:
            annos = coco.loadAnns(coco.getAnnIds(imgIds=im['id']))
            for anno in annos:
                dict[name].append(anno['caption'])

 
# Serializing json
json_object = json.dumps(dict, indent=4)
 
# Writing to man, boy, girl or woman.json
with open("./male_cap.json", "w") as outfile:
    outfile.write(json_object)



loading annotations into memory...
Done (t=0.21s)
creating index...
index created!


# Download images from jsonfile

In [ ]:
from pycocotools.coco import COCO
import requests
import json
base = '/home/cvmlserver/Seohyeon/v-coco/'
coco = COCO('../data/annotations/raw/captions_val2014.json')

with open(os.path.join(base, 'data/annotations/raw/person_image_annotation.json'), 'r') as f:
    data = json.load(f)
    img_list = list(data.keys())
    img_list = [int(img.split('_')[-1].split('.')[0]) for img in img_list]

images = coco.loadImgs(sorted(img_list))

for im in images:
    download = False
    score = 0
    img_data = requests.get(im['coco_url']).content
    with open(base+'/data/original/' + im['file_name'], 'wb') as handler:
        handler.write(img_data)

print('total images downloaded')
